In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


class KPIDataset(Dataset):
    def __init__(self, file_name, window_len=256, sliding_step=1):
        self.start = 0
        self.window_len = window_len
        self.sliding_step = sliding_step
        df = pd.read_csv(file_name, index_col=0, header=0)
        timeseries = ((df.value - df.value.mean()) / df.value.std()).values
        temp = []
        for i in range(self.start, len(timeseries) - self.window_len*self.sliding_step, self.sliding_step):
            temp.append(
                torch.unsqueeze(torch.tensor(timeseries[i:i+self.window_len]), 0)
            )
        self.features = torch.Tensor(len(temp), self.window_len)
        torch.cat(temp, out=self.features)
        #self.features = torch.tensor(self.features)
        #self.end = len(self.timeseries) // self.window_lenght

    def __getitem__(self, i):
        return self.features[i]
    
    def __len__(self):
        return len(self.features)


train_dataset = KPIDataset('./data/1.csv', window_len=64, sliding_step=1)

BATCH_SIZE = 2
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)


In [26]:
class VAE(nn.Module):
    def __init__(self, h_dim=128, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=4, stride=4),
            nn.ELU(),
            nn.Conv1d(16, 32, kernel_size=4, stride=4),
            nn.ELU(),
            nn.Conv1d(32, 64, kernel_size=4, stride=2),
            nn.ELU(),
            nn.Conv1d(64, 128, kernel_size=4, stride=2),
            nn.ELU(),
            nn.Flatten(start_dim=1)
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (h_dim, 1)),
            nn.ConvTranspose2d(h_dim, 64, kernel_size=4, stride=4),
            nn.ELU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=4),
            nn.ELU(),
            nn.ConvTranspose2d(32, 16, kernel_size=4, stride=4),
            nn.ELU(),
            nn.ConvTranspose2d(16, 1, kernel_size=4, stride=4),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar


In [27]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
vae = VAE().to(device)


In [28]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

In [ ]:
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

epochs = 10

for epoch in range(epochs):
    for idx, (x) in enumerate(train_dataloader):
        recon_x, mu, logvar = vae(x)
        loss, bce, kld = loss_fn(recon_x, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        to_print = f"Epoch[{epoch+1}/{epochs}] Loss: {loss.data[0] / BATCH_SIZE} {bce.data[0] / BATCH_SIZE} {kld.data[0] / BATCH_SIZE}")
        print(to_print)
